In [1]:
!pip install Konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 450kB 42.7MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
import konlpy
konlpy.__version__

'0.5.2'

### 파일 업로드

In [6]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving 가성비호텔_도움_라벨링.csv to 가성비호텔_도움_라벨링 (1).csv


In [7]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving korean_stopwords.txt to korean_stopwords.txt


In [9]:
import pandas as pd
df = pd.read_csv('가성비호텔_도움_라벨링 (1).csv')
df.head(3)

,index,hotel,star,review,가성비,추천,출장,조식,무료,저렴,attitude,sum,helpful
0,0,호텔 8 Hours,5,깨끗하고 위치도 좋고 구성비 좋았습니다 다만 비품이 거의 없다고 보면 돼서 챙겨가야...,False,False,False,False,False,False,2,0,0
1,1,호텔 8 Hours,4,너무 좋아요 잘 머물다 갑니다,False,False,False,False,False,False,2,0,0
2,2,호텔 8 Hours,5,적당히 잘 쉬다 가요,False,False,False,False,False,False,2,0,0


### 한글 형태소 분석

In [10]:
from konlpy.tag import Okt
import tqdm.notebook as tn

In [12]:
file = open('korean_stopwords.txt','r',encoding='utf-8')
stop_words = file.readlines()
stop_words = [x.strip() for x in stop_words]
stop_words[:5]

['아', '휴', '아이구', '아이쿠', '아이고']

In [15]:
from konlpy.tag import Okt
okt = Okt()

In [16]:
text = df.review[0]
okt.morphs(text,stem=True)

['깨끗하다',
 '위치',
 '도',
 '좋다',
 '구',
 '성비',
 '좋다',
 '다만',
 '비품',
 '이',
 '거의',
 '없다',
 '보다',
 '돼다',
 '챙기다',
 '하다']

In [17]:
# train-test set 분리
from sklearn.model_selection import train_test_split
X_train_df, X_test_df, y_train, y_test = train_test_split(
    df.review,df.helpful,test_size=0.2,random_state=2021
)

In [18]:
from tqdm import tqdm_notebook

X_train = []
for sentence in tqdm_notebook(X_train_df):
    morphs = okt.morphs(sentence,stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stop_words])
    X_train.append(temp_X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [19]:
X_train[:5]

['좋다 좋다 좋다',
 '깔끔하다 좋다',
 '깔끔하다 깨끗하다',
 '직원 분들 도 친절하다 시설 도 깨끗하다 역시 5 성 급 호텔 이라는 생각 들다 별 3 개 준 이유 는 습격 미니바 때문 이다 미니바 포함 사항 잭 대니엘 코냑 포함 되어다 막상 보다 한 모금 도 안되다 향수병 만 한 잭 대니엘 코냑 속 았 달다 기분 들어서다 썩 기분 좋다 않다 상세 설명 해당 음료 정확하다 용량 표시 하다 필요하다 보이다',
 '항상 쾌적하다']

In [20]:
X_test = []
for sentence in tqdm_notebook(X_test_df):
    morphs = okt.morphs(sentence,stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stop_words])
    X_test.append(temp_X)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [21]:
X_test[:5]

['위치 다 좋다 깔끔하다 샴푸 바디 워시 향 은 개인 적 별로 에요 포근하다 푹 자다 쉬 고 오다 주 차비 만원 넘다 나오다 가시다 분 감안 하다 영등포 위치 는 최고 좋다 조용하다 쉬다 기도 좋다 깔끔하다 좋아하다 추천',
 '좋다',
 '부티크 호텔 이라 방 마다 편차 크다 듯',
 '베스트 오브 베스트 ㅋㅋㅋㅋ',
 '욕조 없다 아쉽다 사진 보이다 방 사이즈 크다 티백 도 신경 써다 고르다 기분 좋다 생수 도 부족하다 프론트 무료 더 주 고요 더 보기']

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, Dropout, MaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer

In [24]:
# 상위 3000개 단어로 인코딩
max_words = 30000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [25]:
X_train[:3]

[[1, 1, 1], [13, 1], [13, 8]]

In [26]:
# 전체 데이터의 길이 분포
print('최대 길이 : ', max(len(l) for l in X_train))
print('평균 길이 : ', sum(map(len,X_train)) / len(X_train))

최대 길이 :  224
평균 길이 :  16.019538602968


In [27]:
max_len = 30
X_train = pad_sequences(X_train,maxlen=max_len)
X_test = pad_sequences(X_test,maxlen = max_len)

In [28]:
model = Sequential([
                    Embedding(max_words,100),
                    LSTM(128),
                    Dense(1,activation="sigmoid")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3000000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 3,117,377
Trainable params: 3,117,377
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=60, epochs=4)

Epoch 1/4
1828/1828 [==============================] - 56s 31ms/step - loss: 0.0729 - accuracy: 0.9781 - val_loss: 0.0561 - val_accuracy: 0.9830
Epoch 2/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0469 - accuracy: 0.9848 - val_loss: 0.0529 - val_accuracy: 0.9837
Epoch 3/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0365 - accuracy: 0.9871 - val_loss: 0.0668 - val_accuracy: 0.9831
Epoch 4/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0275 - accuracy: 0.9901 - val_loss: 0.0735 - val_accuracy: 0.9738


In [32]:
model.evaluate(X_test, y_test)

1071/1071 [==============================] - 3s 3ms/step - loss: 0.0684 - accuracy: 0.9746


[0.06843604147434235, 0.9746103882789612]

In [33]:
from tensorflow.keras.layers import GRU

In [34]:
model2 = Sequential([
                    Embedding(max_words,100),
                    GRU(128),
                    Dense(1,activation="sigmoid")
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         3000000   
_________________________________________________________________
gru (GRU)                    (None, 128)               88320     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 3,088,449
Trainable params: 3,088,449
Non-trainable params: 0
_________________________________________________________________


In [35]:
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])

In [36]:
history2 = model2.fit(X_train, y_train, validation_split=0.2, batch_size=60, epochs=4)

Epoch 1/4
1828/1828 [==============================] - 57s 30ms/step - loss: 0.0699 - accuracy: 0.9790 - val_loss: 0.0520 - val_accuracy: 0.9834
Epoch 2/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0417 - accuracy: 0.9854 - val_loss: 0.0546 - val_accuracy: 0.9835
Epoch 3/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0298 - accuracy: 0.9892 - val_loss: 0.0737 - val_accuracy: 0.9813
Epoch 4/4
1828/1828 [==============================] - 55s 30ms/step - loss: 0.0194 - accuracy: 0.9929 - val_loss: 0.0978 - val_accuracy: 0.9694


In [37]:
model2.evaluate(X_test, y_test)

1071/1071 [==============================] - 3s 3ms/step - loss: 0.0907 - accuracy: 0.9718


[0.0906575620174408, 0.9717504382133484]

In [38]:
from konlpy.tag import Okt
from collections import Counter

In [39]:
# 정규 표현식 함수 정의

import re

def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [40]:
apply_regular_expression(df['review'][0])

'깨끗하고 위치도 좋고 구성비 좋았습니다 다만 비품이 거의 없다고 보면 돼서 챙겨가야 합니다'

In [41]:
corpus = "".join(df['review'].tolist())
corpus[:50]

'깨끗하고 위치도 좋고 구성비 좋았습니다 다만 비품이 거의 없다고 보면 돼서 챙겨가야 합니다'

In [42]:
nouns = okt.nouns(apply_regular_expression(corpus))
print(nouns[:10])

['위치', '성비', '다만', '비품', '거의', '쉬', '가요', '매번', '때', '매우']
